In [2]:
# get the reads 

In [1]:
%%bash 
wget http://nanopore.s3.climb.ac.uk/Zika_iSNV_R9.4_albacore2.3.1.tgz
tar xvfz Zika_iSNV_R9.4_albacore2.3.1.tgz
cat zika-isnv/fast5_r94/workspace/pass/*.fastq > fast5_r94.pass.fastq

--2021-02-05 11:34:30--  http://nanopore.s3.climb.ac.uk/Zika_iSNV_R9.4_albacore2.3.1.tgz
Resolving nanopore.s3.climb.ac.uk (nanopore.s3.climb.ac.uk)... 137.205.52.6
Connecting to nanopore.s3.climb.ac.uk (nanopore.s3.climb.ac.uk)|137.205.52.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1447189312 (1.3G) [binary/octet-stream]
Saving to: ‘Zika_iSNV_R9.4_albacore2.3.1.tgz’

     0K .......... .......... .......... .......... ..........  0%  103K 3h48m
    50K .......... .......... .......... .......... ..........  0%  242K 2h43m
   100K .......... .......... .......... .......... ..........  0%  111M 1h48m
   150K .......... .......... .......... .......... ..........  0%  291K 1h41m
   200K .......... .......... .......... .......... ..........  0%  454K 91m47s
   250K .......... .......... .......... .......... ..........  0%  386K 86m39s
   300K .......... .......... .......... .......... ..........  0%  416K 82m22s
   350K .......... .......... .......... .

In [ ]:
# run deduplication on the reads 

In [2]:
!python scripts/dedup.py fast5_r94.pass.fastq > fast5_r94.dedup.fastq

In [ ]:
# remove adapters

In [3]:
!porechop -b fast5_r94.dedup.porechop -i fast5_r94.dedup.fastq


Loading reads
fast5_r94.dedup.fastq
1,739,084 reads loaded


Looking for known adapter sets
10,000 / 10,000 (100.0%)

Barcodes determined to be in reverse orientation
                                        Best               
                                        read       Best    
                                        start      read end
  Set                                   %ID        %ID     
  SQK-NSK007                               100.0       81.8
  Rapid                                     66.1        0.0
  RBK004_upstream                           81.6        0.0
  SQK-MAP006                                80.0       80.0
  SQK-MAP006 short                          76.9       76.0
  PCR adapters 1                            86.4       79.2
  PCR adapters 2                            82.6       80.0
  PCR adapters 3                            87.0       80.0
  1D^2 part 1                               72.4       75.9
  1D^2 part 2                               87.9    

1,739,084 / 1,739,084 (100.0%)

1,678,788 / 1,739,084 reads had adapters trimmed from their start (120,247,230 bp removed)
1,477,487 / 1,739,084 reads had adapters trimmed from their end (82,388,141 bp removed)


Discarding reads containing middle adapters
1,739,084 / 1,739,084 (100.0%)

5,493 / 1,739,084 reads were discarded based on middle adapters


Saving trimmed reads to barcode-specific files

  Barcode  Reads    Bases        File                               
  BC01     603,555  283,379,825  fast5_r94.dedup.porechop/BC01.fastq
  BC02     698,445  325,134,606  fast5_r94.dedup.porechop/BC02.fastq
  BC03     371,730  174,396,106  fast5_r94.dedup.porechop/BC03.fastq
  none      59,861   30,532,842  fast5_r94.dedup.porechop/none.fastq



In [5]:
# index the ZIKV_REF.fasta reference  
!bwa index reference/ZIKV_REF.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index reference/ZIKV_REF.fasta
[main] Real time: 0.008 sec; CPU: 0.016 sec


In [4]:
!mkdir -p ./bam

In [ ]:
# BC01

In [17]:
%%bash
bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC01.fastq | samtools view -bS - | samtools sort - -o bam/BC01.sorted.bam
samtools index bam/BC01.sorted.bam
python scripts/align_trim.py --normalise 1000 reference/ZikaAsian.scheme.bed < bam/BC01.sorted.bam 2>/dev/null | samtools view -bS - | samtools sort - -o bam/BC01.trimmed.sorted.bam
samtools index bam/BC01.trimmed.sorted.bam
python scripts/freq_SB.py --snpfreqmin 0.03 bam/BC01.trimmed.sorted.bam reference/ZIKV_REF.fasta > BC01.variants.0.03.txt


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 171124 sequences (80000254 bp)...
[M::process] read 170290 sequences (80000226 bp)...
[M::mem_process_seqs] Processed 171124 reads in 279.608 CPU sec, 39.096 real sec
[M::process] read 169704 sequences (80000225 bp)...
[M::mem_process_seqs] Processed 170290 reads in 281.862 CPU sec, 39.682 real sec
[M::process] read 92437 sequences (43379120 bp)...
[M::mem_process_seqs] Processed 169704 reads in 282.326 CPU sec, 39.397 real sec
[M::mem_process_seqs] Processed 92437 reads in 153.410 CPU sec, 27.661 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC01.fastq
[main] Real time: 148.120 sec; CPU: 997.821 sec


In [ ]:
# BC02

In [19]:
%%bash
bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC02.fastq | samtools view -bS - | samtools sort - -o bam/BC02.sorted.bam
samtools index bam/BC02.sorted.bam
python scripts/align_trim.py --normalise 1000 reference/ZikaAsian.scheme.bed < bam/BC02.sorted.bam 2>/dev/null | samtools view -bS - | samtools sort - -o bam/BC02.trimmed.sorted.bam
samtools index bam/BC02.trimmed.sorted.bam
python scripts/freq_SB.py --snpfreqmin 0.03 bam/BC02.trimmed.sorted.bam reference/ZIKV_REF.fasta > BC02.variants.0.03.txt


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 172552 sequences (80000851 bp)...
[M::process] read 171884 sequences (80000748 bp)...
[M::mem_process_seqs] Processed 172552 reads in 278.244 CPU sec, 38.859 real sec
[M::process] read 171522 sequences (80000618 bp)...
[M::mem_process_seqs] Processed 171884 reads in 279.731 CPU sec, 40.411 real sec
[M::process] read 171458 sequences (80000918 bp)...
[M::mem_process_seqs] Processed 171522 reads in 282.706 CPU sec, 39.385 real sec
[M::process] read 11029 sequences (5131471 bp)...
[M::mem_process_seqs] Processed 171458 reads in 281.314 CPU sec, 38.796 real sec
[M::mem_process_seqs] Processed 11029 reads in 18.564 CPU sec, 2.896 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC02.fastq
[main] Real time: 170.882 sec; CPU: 1141.163 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [ ]:
# BC03

In [20]:
%%bash
bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC03.fastq | samtools view -bS - | samtools sort - -o bam/BC03.sorted.bam
samtools index bam/BC03.sorted.bam
python scripts/align_trim.py --normalise 1000 reference/ZikaAsian.scheme.bed < bam/BC03.sorted.bam 2>/dev/null | samtools view -bS - | samtools sort - -o bam/BC03.trimmed.sorted.bam
samtools index bam/BC03.trimmed.sorted.bam
python scripts/freq_SB.py --snpfreqmin 0.03 bam/BC03.trimmed.sorted.bam reference/ZIKV_REF.fasta > BC03.variants.0.03.txt


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 171004 sequences (80000777 bp)...
[M::process] read 170060 sequences (80000398 bp)...
[M::mem_process_seqs] Processed 171004 reads in 280.202 CPU sec, 39.082 real sec
[M::process] read 30666 sequences (14394931 bp)...
[M::mem_process_seqs] Processed 170060 reads in 282.772 CPU sec, 40.582 real sec
[M::mem_process_seqs] Processed 30666 reads in 51.937 CPU sec, 8.221 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -x ont2d -t 8 reference/ZIKV_REF.fasta fast5_r94.dedup.porechop/BC03.fastq
[main] Real time: 88.847 sec; CPU: 615.338 sec
